# Create Functions for Model

## Create Main Function

In [ ]:
def allocate_units_local_for_local(merged_df, target_volume_df, factory_demand_df, constraints=None):
    # Pre-process the data
    merged_df, _ = preprocess_data(merged_df, factory_demand_df, constraints)

    # Calculate total units required
    total_units_required = factory_demand_df['VOLUME'].sum()

    # Initialize the 'allocated_units' column in merged_df
    merged_df['allocated_units'] = 0

    # Start allocation process
    allocation_result = []

    # Iterate through each factory's demand
    for _, factory in factory_demand_df.iterrows():
        factory_id = factory['factory_id']
        factory_country = factory['factory_country']
        required_units = factory['VOLUME']

        # Determine if each vendor is local or not and sort by local_for_local and cost
        merged_df['local_for_local'] = merged_df['vendor_country'] == factory_country
        sorted_vendors = merged_df.sort_values(by=['local_for_local', 'cost_per_unit'], ascending=[False, True])

        # Allocate units to this factory from vendors
        for _, item in sorted_vendors.iterrows():
            vendor_id = item['vendor_id']
            vendor_country = item['vendor_country']
            cost_per_unit = item['cost_per_unit']

            # Find target volume data for this vendor
            target_volume_data = target_volume_df[target_volume_df['vendor_id'] == vendor_id]
            if not target_volume_data.empty:
                target_pct = target_volume_data['target_allocation_pct'].iloc[0]
                acceptable_range_pct = target_volume_data['acceptable_range_pct'].iloc[0] / 100.0
                min_allocation = (target_pct - acceptable_range_pct) * total_units_required
                max_allocation = (target_pct + acceptable_range_pct) * total_units_required

                # Determine potential allocation
                potential_allocation = min(required_units, max_allocation - item['allocated_units'])

                if item['allocated_units'] + potential_allocation >= min_allocation:
                    allocated_units = potential_allocation
                else:
                    allocated_units = max(min_allocation - item['allocated_units'], 0)

                # Update the allocated units for the vendor in sorted_vendors
                merged_df.loc[merged_df['vendor_id'] == vendor_id, 'allocated_units'] += allocated_units
                required_units -= allocated_units

                # Record the allocation details
                if allocated_units > 0:
                    allocation_result.append({
                        'factory_id': factory_id,
                        'vendor_id': vendor_id,
                        'allocated_units': allocated_units,
                        'cost_per_unit': cost_per_unit,
                        'local_for_local': vendor_country == factory_country
                    })

                if required_units <= 0:
                    break

    return pd.DataFrame(allocation_result)


## Create Pre-Processing Function

In [ ]:
def preprocess_data(merged_df, factory_demand_df, constraints=None):
    # Example of filtering based on constraints, if applicable
    if constraints and 'unacceptable_pairs' in constraints:
        for item_id, factory_id in constraints['unacceptable_pairs']:
            merged_df = merged_df[~((merged_df['item_id'] == item_id) & (merged_df['factory_id'] == factory_id))]
    
    # Additional pre-processing logic can be added here

    return merged_df, factory_demand_df
